# Library Implementation
* Transformers: Thư viện phổ biến để làm việc với các mô hình ngôn ngữ lớn như BERT, GPT, LLaMA,...
* LangChain & LangChain Community: Dùng để xây dựng ứng dụng chuỗi tác vụ với LLMs.
* Sentence-Transformers: Dùng để tạo vector embedding chất lượng cao cho truy xuất thông tin và semantic search.
* Wikipedia-API & Underthesea: Hỗ trợ truy xuất và xử lý văn bản tiếng Việt từ Wikipedia và NLP tiếng Việt.
* LlamaIndex: Cung cấp các công cụ để tích hợp LLM với dữ liệu riêng, hỗ trợ truy xuất dựa trên vector store.
* Pandas & python-dotenv: Quản lý dữ liệu và biến môi trường.
* Weaviate Client: Thư viện dùng để kết nối và lưu trữ vector embedding trong cơ sở dữ liệu Weaviate.
* Semantic Chunkers: Dùng để chia tài liệu theo ngữ nghĩa thay vì chỉ dựa vào độ dài token.
* Deepseek SDK: Hỗ trợ tích hợp với mô hình DeepSeek.
* LangChain OpenAI: Giao tiếp dễ dàng giữa LangChain và các API của OpenAI.

In [ ]:
%%capture
!pip install -qU transformers
!pip install -qU langchain langchain-community # Segment
!pip install -qU sentence-transformers
!pip install -q wikipedia-api underthesea # tokenization
!pip install -q llama-index llama-index-vector-stores-weaviate
!pip install -q pandas python-dotenv
!pip install -q weaviate-client # Vector Store
!pip install -q litellm semantic-router semantic-chunkers # Semantic chunking
!pip install -q deepseek-sdk
!pip install -q langchain-openai
!pip install -q pymupdf
!pip install python-docx
!pip install --upgrade "protobuf<6.30.0" "grpcio>=1.72.0"

Xử Lý Dữ Liệu & Tiền Xử Lý
- pandas, numpy, csv, json, os, uuid: Hỗ trợ thao tác dữ liệu, định danh và xử lý file.
- underthesea: Tách từ tiếng Việt.
- spacy: Tách câu và xử lý NLP cổ điển.
- dotenv: Quản lý biến môi trường.
- google.colab.drive: Kết nối Google Drive (chỉ dùng trong môi trường Colab).

Truy Xuất Dữ Liệu & Trích Xuất Nội Dung
- wikipediaapi: Lấy dữ liệu Wikipedia.
- PyPDFLoader, TextLoader (LangChain): Đọc và xử lý file văn bản và PDF.

Mô Hình & Vector Embedding
- transformers: Dùng mô hình ngôn ngữ từ HuggingFace (như BERT, GPT, v.v).
- sentence-transformers: Tạo vector semantic embedding & dùng CrossEncoder.
- llama-index: Tạo hệ thống truy xuất thông tin dựa trên vector index.
- langchain: Kết nối LLM với dữ liệu, xây dựng pipeline hỏi đáp.
- deepseek-sdk: Dùng mô hình DeepSeek nếu cần.

Vector Store & Weaviate
- weaviate, weaviate.classes, weaviate-client: Lưu trữ vector và tìm kiếm tương tự.
- Auth từ weaviate.classes.init: Xác thực kết nối API.

Chia Văn Bản (Chunking / Splitting)
- SentenceSplitter, TokenTextSplitter (LlamaIndex): Chia nhỏ văn bản dựa vào câu hoặc token.
- TextNodeParser, SentenceWindowNodeParser,...: Tùy chọn parser để tạo node trong LlamaIndex.

Mô Hình Ngôn Ngữ & Prompt
- AutoTokenizer, AutoModel: Load tokenizer và mô hình.
- ChatOpenAI, ChatPromptTemplate: Giao tiếp và thiết kế prompt trong LangChain.
- SystemMessage, HumanMessagePromptTemplate: Định dạng lời nhắn đầu vào cho LLM.

Đánh Giá & Hiệu Năng
- tqdm, time: Theo dõi tiến trình.
- multiprocessing, concurrent.futures: Xử lý song song để tăng tốc độ.
- pprint: In đẹp kết quả kiểm thử.


In [ ]:
import json
import weaviate
import csv
import time
import inspect
import llama_index
import numpy as np
import os
import pandas as pd
import spacy
import weaviate.classes as wvc
import wikipediaapi
import uuid
import torch
import requests
import transformers

from pickle import STRING
# Process data
from weaviate.classes.init import Auth
from google.colab import drive
from langchain.document_loaders import PyPDFLoader, TextLoader
from dotenv import load_dotenv
from pprint import pprint
# Chunking, Segment
from underthesea import word_tokenize
from transformers import AutoTokenizer, AutoModel
from llama_index.core.text_splitter import TokenTextSplitter, SentenceSplitter
from llama_index.core import GPTListIndex, Document
from llama_index.core.node_parser.text import *
# evaluation
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
#from sentence_transformers.cross_encoder import losses
from sentence_transformers import SentenceTransformer,CrossEncoder
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# Presequisite information

In [ ]:
class Config:
    DATA_PATH = '/kaggle/input/stock-dataset-qibot'
    PRETRAINED_EMBEDDING_MODEL_PATH = 'bkai-foundation-models/vietnamese-bi-encoder'
    PRETRAINED_RERANKER_MODEL_PATH  = 'Alibaba-NLP/gte-multilingual-reranker-base'
    
    deepseek_api_key = "xxxx"
    wvc_api = 'xxxxx'
    wvc_url = 'xxxxx'
    
    RERANKER_MODEL_NAME = 'Alibaba-NLP/gte-multilingual-reranker-base'

    # TOKEN_TEXT_192s = "Token_text_192"
    # SENTENCEs = "Sentence" 
    # STATISTICALs = "Statistical" 
    REGEXs = "Regex_semantic" # => Use this
    SEMANTIC_CHUNKING = "Semantic_chunking" 
    CONVENTIONAL_CHUNKING = "Conventional_chunking"
    
    collections = {REGEXs} #,TOKEN_TEXT_192s,SENTENCEs,STATISTICALs # => dùng mỗi REGEXs

    lora_r = 16
    lora_alpha = 64
    lora_dropout = 0.1 # 0.05
    target_modules = ['qkv_proj']
    task_type = "SEQ_CLS"
    max_seq_length = 512
    cp = '/kaggle/input/stock-dataset-qibot/checkpoint/checkpoint14588_r16_alpha32_dropout0.1_lr1e3_batch64_mnrLoss' #path pretrained model

    def __init__(self):
        from sentence_transformers import CrossEncoder,SentenceTransformer
        from peft import LoraConfig
        from transformers import AutoModelForSequenceClassification
        self.reranking_model = CrossEncoder(self.RERANKER_MODEL_NAME,
                     max_length=self.max_seq_length,
                     trust_remote_code=True)
        self.backbone_model = AutoModelForSequenceClassification.from_pretrained(self.RERANKER_MODEL_NAME,
                                                                trust_remote_code=True)
        self.lora_config = LoraConfig(
            r=self.lora_r,
            lora_alpha=self.lora_alpha,
            target_modules=self.target_modules,
            lora_dropout=self.lora_dropout,
            bias="none",
            task_type=self.task_type
        )
        self.embedding_model = SentenceTransformer('/kaggle/input/stock-dataset-qibot/model/biencoder_bkai_epoch25_lr64_2e5')
    
# config = Config()

# Connection to Database
* reset_schema(client): Hàm này sẽ xoá toàn bộ schema hiện có trong Weaviate, bao gồm tất cả các collections đã tồn tại. Dùng khi cần khởi tạo lại hệ thống từ đầu.
* create_schema(client, collections)

Hàm này tạo mới các collections trong Weaviate với các cấu hình sau:
* Vectorizer: Sử dụng OpenAI để sinh vector embedding (text2vec-openai).
* Generative Model: Dùng Cohere để hỗ trợ các truy vấn sinh văn bản (generative search).

Các thuộc tính trong mỗi collection gồm:
* source: Nguồn gốc của văn bản.
* chunk_id: ID duy nhất cho mỗi đoạn văn bản.
* metadata: Mảng metadata đi kèm (dùng cho filter hoặc hiển thị).
* content: Nội dung văn bản gốc.

In [ ]:
class Client: 
    def __init__(self, config,collection):
        self.cfg = config
        
        self.client = weaviate.connect_to_weaviate_cloud(
          cluster_url=self.cfg.wvc_url,
          auth_credentials=Auth.api_key(self.cfg.wvc_api),
          headers={}
          #headers={'X-OpenAI-Api-key': OPENAI_API_KEY}
        )
        self.collection = collection
        
    def get_cluster(self, collection_name):
        return self.client.collections.get(collection_name)

    def connect(self):
        self.client.connect()
        
    def close(self):
        self.client.close()
    
    def is_ready(self):
        return self.client.is_ready()
    
    def reset_schema(self):
        """
        Xóa toàn bộ schema (collections) hiện tại trong Weaviate client.
        """
        self.client.connect()
        self.client.collections.delete_all()
        self.client.close()
        
    
    def create_schema(self, collections):
        """
        Tạo mới các schema (collections) trong Weaviate với cấu hình cụ thể.
        
        Args:
            client: Weaviate client đã kết nối.
            collections: Danh sách tên các collections cần tạo.
        
        Mỗi collection được cấu hình với:
        - Vectorizer: OpenAI text embedding
        - Generator: Cohere LLM
        - Các thuộc tính: document_id, stock_id, metadata, text
        """
        self.client.connect()
        try:
            for collection_name in collections:
                docs = self.client.collections.create(
                  name = collection_name,
                  vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
                  generative_config=wvc.config.Configure.Generative.cohere(),
                  properties=[
                      wvc.config.Property(name="source",data_type=wvc.config.DataType.TEXT),
                      wvc.config.Property(name="chunk_id",data_type=wvc.config.DataType.TEXT),
                      wvc.config.Property(name="metadata",data_type=wvc.config.DataType.TEXT_ARRAY),
                      wvc.config.Property(name="content",data_type=wvc.config.DataType.TEXT)]
                )
                print(f"Đã tạo {collection_name} thành công")
        except Exception as e:
            print(f"❌ Lỗi khi tạo schema: {e}")
        self.client.close()

In [ ]:
def insert_batch(objs, client, collection_name):
  BATCH_SIZE = 256
  cluster = client.get_cluster(collection_name)

  for i in range(0, len(objs), BATCH_SIZE):
    batch = objs[i:i + BATCH_SIZE]
    cluster.data.insert_many(batch)
    print(f"✅ Batch {i + 1} to {i + len(batch)} inserted successfully!")

  print(f"Data insertion into {collection_name} completed!\n")

# Embedding model

In [ ]:
from sentence_transformers import SentenceTransformer,CrossEncoder,util

def gen_embedding(contents):
    contents = contents.strip().lower()
    return embedding_model.encode(contents).tolist()

embedding_model = config.embedding_model
reranker_model = config.reranking_model

# Tokenization

In [ ]:
def segment(segment_text, format_ = "text"):
  return word_tokenize(segment_text, format_).strip().lower()
segment("Tuan Anh là Sinh vien trường đại học công nghệ")

# Chunker

In [ ]:
from semantic_chunkers.chunkers.regex import RegexChunker
regex_chunker = RegexChunker()

# Preprocessing Data

In [ ]:
import os
import uuid
import fitz
from pathlib import Path
import tiktoken

In [ ]:
def count_tokens(text: str,
                 model: str = "gpt-3.5-turbo"):
  encoding = tiktoken.encoding_for_model(model)
  tokens = encoding.encode(text)
  return len(tokens)
    
def handle_keyword(text):
    from underthesea import pos_tag
    # Tokenize & gán nhãn từ loại
    tagged_words = pos_tag(text)

    # Lọc danh từ (Từ loại "N" là danh từ)
    nouns = [word.lower() for word, pos in tagged_words if pos == "N"]

    count_map = {}
    for item in nouns:
        count_map[item] = count_map.get(item, 0) + 1

    filtered_counts = {key: value for key, value in count_map.items() if value > 1 and " " in key}

    sum_tokens = 0
    for item in filtered_counts:
      sum_tokens += filtered_counts.get(item)

    avg_count = sum_tokens / len(filtered_counts) if filtered_counts else 0

    # Lọc những danh từ có số lần xuất hiện > trung bình
    filtered_nouns = [word.lower() for word, count in filtered_counts.items() if count > avg_count]

    return filtered_nouns

In [ ]:
def process_file(file_path):
    """Load file content based on extension (.pdf or .txt)."""
    source = Path(file_path).stem
    ext = Path(file_path).suffix.lower()

    if ext == ".pdf":
        with fitz.open(file_path) as doc:
            content = "".join(page.get_text() for page in doc)
    elif ext == ".txt":
        loader = TextLoader(file_path).load()
        content = loader[0].page_content
    else:
        print(f"Unsupported file type: {file_path}")
        return []
    
    return [{"source": source, "content": content}]

def format_chunk(chunks):
  objs = []
  for chunk in chunks:
    text = (" ".join(chunk.splits))
    # keywords = handle_keyword(text_)
    content = {
    'metadata': handle_keyword(text), # text_array
    'content': segment(text)
    }
    objs.append(content)
  return objs

def process_data(corpus, chunker=RegexChunker()):
  objs = [] # list
  for obj in corpus:
    metadata=obj.get('metadata', "default")
    source=obj.get('source', "default")
    if not isinstance(metadata, list):
        metadata = [metadata]
        
    text = obj['content']
    chunks = format_chunk((chunker([text]), metadata)[0][0]) # Chỉ dùng cho RegexChunker
      
    for chunk in chunks:
      content = chunk['content'].strip().lower()
      objs.append(
          wvc.data.DataObject(
              properties={
                  "source": source,
                  "chunk_id": str(uuid.uuid4()),
                  "metadata": metadata,
                  "content": content
              },
              vector=gen_embedding(content)
          )
    )
  print(f"Total chunks: {len(objs)}")
  return objs

def load_data(path):
    docs = []
    
    if path.startswith("http://") or path.startswith("https://"):
        return process_url(path)

    elif os.path.isfile(path):
        docs.extend(process_file(path))

    elif os.path.isdir(path):
        for file in os.listdir(path):
            file_path = os.path.join(path, file)
            if os.path.isfile(file_path):
                docs.extend(process_file(file_path))
    if not docs:
        print(f"Không tìm thấy tài liệu nào trong đường dẫn: {path}")
    return docs

# Retrieval

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def normalize_bi_encoder(scores):
    return ((np.array(scores) + 1) / 2)

def normalize_cross_encoder(scores):
    return 1 / (1 + np.exp(-np.array(scores)))

def hybrid_score_fusion(bi_scores, cross_scores, alpha=0.4):
    bi_norm = normalize_bi_encoder(bi_scores)
    cross_norm = normalize_cross_encoder(cross_scores)
    return alpha * bi_norm + (1 - alpha) * cross_norm
        
class Retrieval: 
    def __init__(self, config, cluster):
        self.cfg = config
        self.cluster = cluster
        self.collection_name = config.REGEXs
        self.top_k = config.TOP_K
        self.hybrid_factor = config.HYBRID_FACTOR
        self.alpha = config.ALPHA   
        self.reranker_model = config.reranking_model
        
    def keyword_retrieval(self, query):
      response = self.cluster.query.bm25(
        query=segment(query),
        limit=top_k,
        return_metadata=["score"]
      )
      if response.objects is None:
        print(f"Error retrieval() return None")
        return []
      return response.objects

    def hybrid_retrieval(self, query, top_k=self.top_k, hybrid_factor=self.hybrid_factor):
        segment_query = segment(query)
        response = self.cluster.query.hybrid(
            query=segment_query,
            vector=gen_embedding(segment_query),
            alpha=0.5,
            limit=top_k * hybrid_factor,
            return_metadata=["score"],
        )
         # Loại bỏ kết quả trùng text
        seen = set()
        results = []
        for obj in response.objects:
            text = obj.properties.get("content", "").strip()
            if not text or text in seen:
                continue
            seen.add(text)
            results.append(obj)
            if len(results) == top_k:
                break
        
        return results

    def rerankce_retrieval(self, query, top_k=self.top_k, hybrid_factor=self.hybrid_factor):
        # Lấy nhiều hơn top_k từ bi-encoder (hybrid retrieval)
        retrieved_docs = self.hybrid_retrieval(query)
        if not retrieved_docs:
            return [], [], [], []
    
        bi_encoder_scores = np.array([doc.metadata.score for doc in retrieved_docs])
        
        segment_query = segment(query)
        passage_pairs = [(segment_query, doc.properties['content']) for doc in retrieved_docs]
        cross_encoder_scores = np.array(self.reranking_model.predict(passage_pairs))
        
        fused_scores = hybrid_score_fusion(bi_encoder_scores, cross_encoder_scores, self.alpha)
        
        top_indices = heapq.nlargest(top_k, range(len(fused_scores)), key=fused_scores.__getitem__)
        
        # Chuẩn bị kết quả
        indices = [idx+1 for idx in top_indices]
        docs = [retrieved_docs[idx] for idx in top_indices]
        cross_scores = [float(cross_encoder_scores[idx]) for idx in top_indices]
        fused_scores = [float(fused_scores[idx]) for idx in top_indices]
    
        return indices, cross_scores, fused_scores, docs

In [ ]:
import os
OPENAI_API_KEY = 'sk-proj-xxxxxx'
# os.environt['OPENAI_API_KEY'] = OPENAI_API_KEY

RESPONSE_PROMPT = """
Bạn là chuyên gia tư vấn về chứng khoán và thị trường tài chính.
Dựa trên câu hỏi: {query} và ngữ cảnh: {context}, hãy trả lời chính xác, chi tiết, rõ ràng và tự nhiên bằng Tiếng việt.
Nếu thiếu thông tin để đưa ra kết luận chính xác, hãy yêu cầu thêm dữ liệu thay vì suy đoán."""

def gen_response(doc):
    try:
      query = doc['question']
      retrieved_chunks = doc['retrieved_chunks']
      context = "\n".join([doc.get()for doc in retrieved_chunks])
      print(f"Content\n{context} \n")
      documents = [Document(text=context)]
      index = GPTListIndex.from_documents(documents)
      query_engine = index.as_query_engine()
      prompt = RESPONSE_PROMPT.format(query=query, context=context)
      response = query_engine.query(prompt)
      return response.response # str
    except Exception as e:
        return e
        
# gen_response(regex[0])

# Demo

In [ ]:
config = Config()
client = Client(config,config.REGEXs)
cluster = client.get_cluster(config.REGEXs)
print(client.is_ready())
client.reset_schema()
client.create_schema(config.collections)
folder_path=f"{config.DATA_PATH}/knowledge"
path_data = insert_batch(process_data(load_data(folder_path), regex_chunker), client, config.REGEXs)
retrieval = Retrieval(config,cluster)

query = 'Chung khoan la gi'
context = retrieval.rerankce_retrieval(query)
gen_response(context)